In [57]:
#Task 1
#Reading the data
import pandas as pd
dataset=pd.read_csv("assignment.txt", delimiter="\t", names = ["Tweet index", "Label", "Tweet text"])
dataset = dataset.set_index("Tweet index")
df = dataset.drop(dataset.index[0])
print(df.head())
X,Y = df.iloc[:,-1].values, df.iloc[:,0].values


            Label                                         Tweet text
Tweet index                                                         
1               1  Sweet United Nations video. Just in time for C...
2               1  @mrdahl87 We are rumored to have talked to Erv...
3               1  Hey there! Nice to see you Minnesota/ND Winter...
4               0                3 episodes left I'm dying over here
5               1  I can't breathe! was chosen as the most notabl...


In [60]:
import nltk
import re
for i in range(len(X)):
    X[i] = X[i].lower()
    X[i] = re.sub(r'\W', ' ', X[i]) #removing whitespace
    X[i] = re.sub(r'\s+', ' ', X[i]) #removing punchuations
X

array(['sweet united nations video just in time for christmas imagine noreligion http t co fej2v3oubr',
       ' mrdahl87 we are rumored to have talked to erv s agent and the angels asked about ed escobar that s hardly nothing ',
       'hey there nice to see you minnesota nd winter weather ', ...,
       'well now that i ve listened to all of into the woods i m going to listen to some fob nosurprisethere',
       'hummingbirds are experts at hovering after all background motion if the hovering ability of the http t co e189ihbpzr',
       'only thing missing now is a session at the gym want to do every body part though and cv possible'],
      dtype=object)

In [72]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

wordfreq = {}
for sentence in X:
    tokens = nltk.word_tokenize(sentence) #tokenising the sentences
    tokens = [word for word in tokens if word not in stop_words] #removing the stop words #if not 12768 or 12624
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print(len(wordfreq))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


12624


In [78]:
df.groupby('Label').count()

,Tweet text
Label,
0,1916
1,1901


In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 2/3, shuffle = True)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(2544,) (1273,) (2544,) (1273,)


In [89]:
def evaluation(predicted, actual):
    true_pos, true_neg, false_pos, false_neg = 0, 0, 0, 0
    for p,g in zip(predicted, actual):
        if p==1 and g==1:
            true_pos+=1
        if p==0 and g==0:
            true_neg+=1
        if p==1 and g==0:
            false_pos+=1
        if p==0 and g==1:
            false_neg+=1
        
    precision = true_pos / float(true_pos + false_pos)
    recall = true_pos / float(true_pos + false_neg)
    f1_score = 2*precision*recall / (precision + recall)
    accuracy = (true_pos + true_neg) / float(len(actual))
    
    return accuracy, precision, recall, f1_score

#a,p,r,f = evaluation([1,0,0,1,0], [1,1,0,0,0])
#print(a, p, r, f)

0.6 0.5 0.5 0.5


In [90]:
#log linear function
def sigmoid(X, weight):
    z = np.dot(X, weight)
    return 1 / (1 + np.exp(-z))

def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]

def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient